# Pure mutation version

In [ ]:
import cv2
import numpy as np
import math
import random
import time
import os
import sys

In [ ]:
cv2.__version__

'4.10.0'

In [ ]:
channel = 2  # B + alpha
IMG_SIZE = 100
# how many points in each unit of chronosome
unit_shape_points = 3
# amount of chronosome
unit_num = 100

In [ ]:
class GA():
  iter_num = 0
  target_img = None

  def __init__(self):
    self.fitness = 0
    self.unit_points = np.random.randint(
        0, IMG_SIZE,
        (unit_num, unit_shape_points, 2)
    )
    self.unit_channels = np.random.randint(
        0, 256, (unit_num, channel)
    )

  def draw_whole_unit(self):
    self.units = np.ascontiguousarray(
        np.full(
            (unit_num, IMG_SIZE, channel),
            0,
            dtype=np.int8
        )
    )
    for i in range(unit_num):
      # polygon
      cv2.fillPoly(
          self.units[i],
          [self.unit_points[i]],
          tuple(self.unit_channels.tolist())
      )

    # concatinate all chronosome
    def draw(self):
      # background_color is white
      self.img = np.full(
          (IMG_SIZE, IMG_SIZE, channel-1), 255
      )
      for i in range(unit_num):
        alpha = self.units[i, :, :, -1] / 255.0
        alpha = np.array([alpha])
        alpha = np.swapaxes(alpha, 0, 1)
        alpha = np.swapaxes(alpha, 1, 2)
        self.img[:, :, :channel-1] = self.units[
            i, :, :, :channel-1
        ]*alpha + self.img[
            :, :, :channel-1
        ]*(np.ones(alpha.shape) - alpha)

    def fast_draw(self, unit_list):
      img = np.full((IMG_SIZE, IMG_SIZE, channel-1), 255)
      for unit in unit_list:
        alpha = unit[:, :, -1] / 255.0
        alpha = np.array([alpha])
        alpha = np.swapaxes(alpha, 0, 1)
        alpha = np.swapaxes(alpha, 1, 2)
        img[:, :, :channel-1] = unit[
            :, :, :channel-1
        ]*alpha + img[
            :, :, :channel-1
        ]*(np.ones(alpha.shape) - alpha)
        return img

    # calculate fitness with L2 norm
    def fit(self, img=None):
      if img is None:
        compare_img = self.img
      else:
        compare_img = img
      sub = (self.target_img - compare_img).astype(np.int64)
      sum_ = np.sum(sub*sub)
      fitness = np.sqrt(sum_)
      return fitness

    def variation(self):
      var_rate_list = [0.3, 0.1, 0.05, 0.03, 0.02, 0.01, 0.01]
      iter_list = [-1, 50, 100, 200, 400, 600, 800]

      for i in range(len(var_rate_list)-1, -1, -1):
        if self.iter_num > iter_list[i]:
          var_rate = var_rate_list[i]
          break

      var_num = int(unit_num * var_rate)
      random_list = random.sample(range(unit_num), var_num)

      for iter in range(10):
        change_unit_points = self.unit_points.copy()
        change_unit_channels = self.unit_channels.copy()
        change_units = self.units.copy()
        for i in random_list:
          if var_rate == var_rate_list[-3]:
            change_unit_points[i] = self.var_points(i, 15)
            change_unit_channels[i] = self.var_color(i, 30)
          elif var_rate == var_rate_list[-2]:
            change_unit_points[i] = self.var_points(i, 5)
            change_unit_channels[i] = self.var_color(i, 15)
          elif var_rate == var_rate_list[-1]:
            change_unit_points[i] = self.var_points(i, 5)
            change_unit_channels[i] = self.var_color(i, 15)
          else:
            change_unit_points[i] = np.random.randint(
                0, IMG_SIZE, (unit_shape_points, 2)
            )
            change_unit_channels[i] = np.random.randing(0, 255, (channel))

          change_units[i] = np.random.randing(
              IMG_SIZE, IMG_SIZE, channel
          )
          cv2.fillPoly(
              change_units[i],
              [change_unit_points[i]],
              tuple(change_unit_channels[i].tolist())
          )

        img = self.fast_draw(change_units)
        fitness_ = self.fit(img)
        if fitness_ < self.fitness:
          self.fitness = fitness_
          self.img = img
          for i in random_list:
            self.unit_points[i] = change_unit_points[i]
            self.unit_channels[i] = change_unit_channels[i]
            self.units[i] = change_units[i]
          break

  def target_format(self, target_img):
    if target_img.shape[:2] != (IMG_SIZE, IMG_SIZE):
      target_img = cv2.resize(target_img, (IMG_SIZE, IMG_SIZE))
    if channel == 2:
      if not target_img.shape[-1] == 1:
        target_img = cv2.cvtColor(target_img, cv2.COLOR_BGR2GRAY)
        target_img = np.array([target_img])
        target_img = np.swapaxes(target_img, 0, 1)
        target_img = np.swapaxes(target_img, 1, 2)
    elif not channel == 4:
      try:
        sys.exit(0)
      except:
        print("wrong channels")

    if not self.img.shape == target_img.shape:
      try:
        sys.exit(0)
      except:
        print("target shape not match")
    self.target_img = target_img

  def var_points(self, ele, val):
    point = np.zeros((unit_shape_points), 2)
    for i in range(unit_shape_points):
      point[i] = np.array([
          min(
              max(0, np.random.randint(-val, val) +\
                  self.unit_points[ele][i][0]),
              IMG_SIZE
          ),
          min(
              max(0, np.random.randing(-val, val) +\
                  self.unit_points[ele][i][1]),
              IMG_SIZE
          )
      ])
    return point

  def var_color(self, ele, val):
    i_channel = np.zeros((channel))
    for i in range(channel):
      i_channel[i] = min(
          max(0, np.random.randint(-val, val) +\
              self.unit_channels[ele][i]),
          255
      )
    return i_channel